Copyright 2022 AudioT.  All rights reserved.
This code is proprietary to AudioT, INC and provided to OMSA students under the conditions of the AudioT NDA terms. 

**NOTE:** This walkthrough was written up by a previous practicum student using a previous version of the `audio_features_example.py`, so some differences may exist between it and the current version.

# Audio Features Example Walkthrough

This is a detailed explanation of the code contained in the file `audio_features_example.py`. The code from that example is reproduced below with each step explained. Additional code is added so the results at each step can be explored.

The overall process of this example is the following:
1. [Load the signal data](#1.-Load-the-signal-data) (in an AudioSignal object)
2. [Calculate log Mel energy features](#2.-Calculate-log-Mel-energy-features) (in an AudioFeatures object)
3. [Load event labels](#3.-Load-event-labels) (in a pandas DataFrame)
4. [Match event labels to features](#4.-Match-event-labels-to-features) (in a Numpy array in  the AudioFeatures object)
5. [Plot labels and features together](#5.-Plot-labels-and-features-together) (using matplotlib)

First, the required packages are imported. Note that a number of utility functions are imported from the **audiot** folder.

In [ ]:
# I added these 2 lines because __file__ does not exist in Jupyter notebook
import os.path
__file__ = os.path.abspath('')

import sys
from pathlib import Path
# Add the main project directory (parent folder) to sys.path so we can import our project modules
project_folder = Path(__file__).absolute().parent
sys.path.append(str(project_folder))

import librosa
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp

from audiot.audio_features import AudioFeatures, calc_log_mel_energy_features
from audiot.audio_signal import AudioSignal
from audiot.audio_labels import load_labels

## 1. Load the signal data
Locate the file of interest and its label file. Place them in a directory called **test_data** that's at the same level as the **examples** directory this file is in.

In [ ]:
file_path = project_folder / "test_data/LT3-G3_2014-11-05_15.58.00_ch2.flac"
label_path = project_folder / "test_data/LT3-G3_2014-11-05_15.58.00_ch2_label.txt"

Create an AudioSignal object for this audio data. Do this using the `AudioSignal.from_file()` method defined in `audio_signal.py`. This method uses the `soundfile` package to read the waveform and several of its properties. The `AudioSignal` class then stores all this data including: waveform, sample rate, file path, file format, and sub-type. 

In [ ]:
signal = AudioSignal.from_file(file_path)

print(type(signal))

In [ ]:
from pprint import pprint
pprint(signal.__dict__)

The signal waveform data is stored in a Numpy array called `signal`. Several other dynamic properties are computed fromm the shape of this data.

In [ ]:
print(signal.n_samples)

In [ ]:
print(signal.n_channels)

Note that the file has a sample rate of 48kHz samples/sec and contains a total of 2880 ksamples. Therefore, the audio file has a duration of 2880/48 = 60 seconds.

## 2. Calculate log Mel energy features
The next step is to take the raw waveform data and use it to compute features in the frequency domain. This is all done within the `calc_log_mel_energy_features()` function defined in `audio_features.py`. This function does the following

* Define number of samples per bin as `n_fft` given a desired window length (default is 50 ms)
* Define distance between starts of consecutive bins as `hop_length` (default is half of `n_fft` for 50% overlap)
* Use the [`librosa`](https://librosa.github.io/librosa/generated/librosa.feature.melspectrogram.html) library to compute the mel energy spectogram for a specified `n_fft`, `hop_length`, frequency range, and number of Mel bands
* Take the log to make these log Mel energy features
* Compute the start time of each bin in seconds based on the discretization details
* Store the Mel spectrogram and timing details in an AudioFeatures object using the `AudioFeatures` class defined in `audio_features.py`

In [ ]:
features = calc_log_mel_energy_features(signal)

print(type(features))

Define the names of the events in this example. We are only interested in _cough_ events, so others will be ignored.

In [ ]:
features.event_names = ["cough"]

Let's take a look at the attributes contained in the AudioFeatures object `features`. As mentioned above, this object stores the log Mel energies as a 2D Numpy array. It also stores the frame start times and durations as well as the event names we are interested in.

In [ ]:
pprint(features.__dict__)

The features are stored in a 2D Numpy array called `features`. Other attributes are computed from the shape of this array.

In [ ]:
print(features.n_features) # same as features.features.shape[1]

In [ ]:
print(features.n_samples) # same as features.features.shape[0]

The array of log Mel energies has 40 columns, one for each Mel band. It also has 2401 rows, one for each start time. Does this make sense? We had 60 seconds of data and wanted to look at them in 50 ms windows with a 50% overlap. Therefore, every start time is 25 ms apart. The number of windows is $\frac{60s}{25ms} = 2400$. There is one extra window because of the half-window overlap at the beginning and end of the clip.

## 3. Load event labels
We now have the signal data stored in an AudioSignal object, and the features data stored in an AudioFeatures object. The next step is to load the labels from the labels text file. This is done using the `load_labels` function from `audio_labels.py`. This function reads the labels from the `.txt` file into a pandas dataframe. The dataframe has columns for the event onset and offset as well as the label. It also internally calls `audio_labels.clean_overlapping_labels()` to merge identical adjacent or overlapping labels.

In [ ]:
labels = load_labels(label_path)
labels.head()

## 4. Match event labels to features

Next, compare the time windows in the `labels` dataframe with the time windows where the log Mel energy features were defined. For every time window in the Mel features, determine if the event occured or not. The `match_labels()` method of the `AudioFeatures` class does this in the following steps:
* Initialize a Numpy array `events` with zeros. It has one row for each Mel sample and one column for each event type (In this case, we only care about _cough_ events.
* Define an overlap threshold for when to say an event occured in a given Mel sample window (default is 50%)
* For each event labeled _cough_, find all the Mel sample windows that overlap with the times where the event was detected.
* For every overlapping sample found, store a 1 in the `events` array.
* Store the `events` array in the `true_events` attribute of the AudioFeatures object (in our case, it's the `features` variable)

In [ ]:
features.match_labels(labels)

print(features.true_events)

In [ ]:
print(features.true_events.shape)

## 5. Plot labels and features together
The final step in this example is plotting the data. Before plotting, the data is combined into a single array. This array contains both the log Mel energy features data as well as the true events data. This way, the features and true events can be visually compared in the same plot.

When stacking these arrays, they are first transposed so that the time will be on the x-dimension of the plot. The true events array values are also scaled from [0,1] to the range of values of the feature array. This ensures that the colormap of the resulting plot will be consistent for both types of data we are representing.

In [ ]:
min_val = np.amin(features.features)
max_val = np.amax(features.features)
plot_data = np.vstack(
    (features.features.T, features.true_events.T * (max_val - min_val) + min_val)
)

print(features.features.T.shape)

In [ ]:
print(features.true_events.T.shape)

In [ ]:
print(plot_data.shape)

To plot the data, use the standard set of `matplotlib` functions to display and annotate the data. Specifically, the `imshow()` function is used for visualizing the `plot_data` array as an image. The `extent` argument allows us to use time information rather than index for positioning the image on the x-axis. Other `matplotlib` functions are used to add a title, xlabel, ylabel, and colorbar.

In [ ]:
plt.figure()
plt.imshow(
    plot_data,
    interpolation="nearest",
    aspect="auto",
    origin="lower",
    extent=(features.frame_start_times[0], features.frame_end_times[-1], 0, plot_data.shape[0]),
)
plt.title("Log Mel energy spectrum")
plt.xlabel("Time (seconds)")
plt.ylabel("Frequency band")
plt.colorbar()

plt.show()